## Preprocessing

In [87]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [88]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [89]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [90]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [91]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [92]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [93]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts.loc[classification_counts > 1]
classification_counts_gt1.head()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: count, dtype: int64

In [94]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [95]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [96]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [97]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [98]:
# FIRST ATTEMPT -- ACCURACY 73%
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 8)                 352       
                                                                 
 dense_38 (Dense)            (None, 5)                 45        
                                                                 
 dense_39 (Dense)            (None, 1)                 6         
                                                                 
Total params: 403 (1.57 KB)
Trainable params: 403 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [99]:
# Compile the model (First Attempt)
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [100]:
# Train the model (First Attempt Accuracy 73%)
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6131 - accuracy: 0.6885
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5640 - accuracy: 0.7234
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5588 - accuracy: 0.7273
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7281
Epoch 5/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7272
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5535 - accuracy: 0.7287
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7305
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7314
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7310

In [101]:
# Evaluate the model using the test data (First Attempt)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5628 - accuracy: 0.7247 - 634ms/epoch - 2ms/step
Loss: 0.5628142356872559, Accuracy: 0.7246647477149963


In [111]:
# SECOND ATTEMPT -- ACCURACY 73%
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 10)                440       
                                                                 
 dense_49 (Dense)            (None, 8)                 88        
                                                                 
 dense_50 (Dense)            (None, 6)                 54        
                                                                 
 dense_51 (Dense)            (None, 1)                 7         
                                                                 
Total params: 589 (2.30 KB)
Trainable params: 589 (2.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [112]:
# Compile the model (Second Attempt)
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [113]:
# Train the model (Second Attempt Accuracy 73%)
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6276 - accuracy: 0.6750
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5718 - accuracy: 0.7320
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5676 - accuracy: 0.7328
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5631 - accuracy: 0.7331
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7331
Epoch 6/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5562 - accuracy: 0.7321
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7321
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7319
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7333
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7339

In [105]:
# Evaluate the model using the test data (Second Attempt)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5586 - accuracy: 0.7214 - 492ms/epoch - 2ms/step
Loss: 0.5585665106773376, Accuracy: 0.7213994264602661


In [106]:
# THIRD ATTEMPT -- ACCURACY 73%
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 12)                528       
                                                                 
 dense_45 (Dense)            (None, 10)                130       
                                                                 
 dense_46 (Dense)            (None, 8)                 88        
                                                                 
 dense_47 (Dense)            (None, 1)                 9         
                                                                 
Total params: 755 (2.95 KB)
Trainable params: 755 (2.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [107]:
# Compile the model (Third Attempt)
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [114]:
# Train the model (Third Attempt Accuracy 73%)
fit_model = nn.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7353
Epoch 2/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7348
Epoch 3/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7360
Epoch 4/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7341
Epoch 5/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7360
Epoch 6/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7362
Epoch 7/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7356
Epoch 8/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7353
Epoch 9/30
804/804 [==============================] - 1s 2ms/step - loss: 0.5456 - accuracy: 0.7353
Epoch 10/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7368

In [115]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7255 - 439ms/epoch - 2ms/step
Loss: 0.5582599639892578, Accuracy: 0.7254810333251953


In [116]:
# Export our model to HDF5 file
nn.save('/content/AlphabetSoupCharity_Optimization.h5')

files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>